# CNN_1 Basic Model with NB-BoW Baseline and comparable CNN

In [91]:
from __future__ import division
import os, sys, re, json, time, datetime, shutil, csv
import itertools, collections
from importlib import reload
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk
from nltk.tokenize import word_tokenize

# NumPy and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1.8"))




In [92]:
# Load labeled data set
review_sentences = pd.read_csv('./data/Feature_sentences_Prod_Final_2018-07-09_Labeled_1K.csv', encoding='utf-8')
# Distribution by Sentiment Class
print(review_sentences.groupby('POS_SENTIMENT').size())

POS_SENTIMENT
0    213
1    826
dtype: int64


In [93]:
# Create training and test sets, keeping 80:20 ratio of sentiment class
dataset_class_0 = review_sentences[review_sentences['POS_SENTIMENT'] == 0]
dataset_class_1 = review_sentences[review_sentences['POS_SENTIMENT'] == 1]

#train = 170:660 = 830
#test = 43:166 = 209
#               == 1,039

train = dataset_class_0.iloc[0:170,]
train = train.append(dataset_class_1.iloc[0:660,])
print('train shape:', train.shape)

test = dataset_class_0.iloc[170:213,]
test = test.append(dataset_class_1.iloc[660:826,])
print('test shape:', test.shape)

train shape: (830, 4)
test shape: (209, 4)


In [94]:
train_x = train['SENTENCE']
train_y = train['POS_SENTIMENT']
test_x = test['SENTENCE']
test_y = test['POS_SENTIMENT']

In [95]:
# Tokenize sentences

train_x_tokens = []
test_x_tokens = []

for x in train_x:
    train_x_tokens.append(word_tokenize(x))
    
for x in test_x:
    test_x_tokens.append(word_tokenize(x))

In [96]:
# define vocabulary using w266 common vocab

train_text_all = [item for sublist in train_x_tokens for item in sublist]

vocab = vocabulary.Vocabulary(train_text_all, size=20000)
print("Vocabulary size: {:,}".format(vocab.size))
#print("Vocabulary dict: ", vocab.word_to_id)

train_x_ids = []
test_x_ids = []

for x in train_x_tokens:
    train_x_ids.append(vocab.words_to_ids(x))
    
for x in test_x_tokens:
    test_x_ids.append(vocab.words_to_ids(x))

Vocabulary size: 1,699


In [97]:
# NB - BoW Baseline

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

train_x_fdict = []
test_x_fdict = []

for x in train_x_ids:
    train_x_fdict.append(collections.Counter(x))
    
for x in test_x_ids:
    test_x_fdict.append(collections.Counter(x))

train_x_vector = []
test_x_vector = []

num_features = vocab.size
print('creating id lists...')
for x in train_x_fdict:
    train_x_vector.append([x.get(i, 0) for i in range(num_features)])
    
for x in test_x_fdict:
    test_x_vector.append([x.get(i, 0) for i in range(num_features)])

# use w266 common utils to convert id lists to sparse bow matrix
print('starting id lists to sparse bow conversion...')
train_x_sparse_bow = utils.id_lists_to_sparse_bow(train_x_fdict, vocab.size)
test_x_sparse_bow = utils.id_lists_to_sparse_bow(test_x_fdict, vocab.size)

# training Multinomial Naive Bayes for simple baseline model
print('training Multinomial Naive Bayes for simple baseline model...')
nb = MultinomialNB()
nb.fit(train_x_sparse_bow, train_y)
y_pred = nb.predict(test_x_sparse_bow)

acc = accuracy_score(test_y, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))
classrep = classification_report(test_y, y_pred)
print(classrep)

creating id lists...
starting id lists to sparse bow conversion...
training Multinomial Naive Bayes for simple baseline model...
Accuracy on test set: 84.21%
             precision    recall  f1-score   support

          0       0.69      0.42      0.52        43
          1       0.86      0.95      0.91       166

avg / total       0.83      0.84      0.83       209



In [98]:
# Preprocessing for CNN model

from sklearn import preprocessing
# pad all id vectors to max length for Neural model

max_comment_length = max([len(x) for x in train_x_ids])

train_x_neural = train_x_ids
test_x_neural = test_x_ids

for x in train_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]
        
for x in test_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]

# convert labels to one-hot vectors for Neural model 

terms_list = [[0],[1]]
lb = preprocessing.MultiLabelBinarizer()
lb.fit(terms_list)

train_labels = np.array(train_y.tolist())
test_labels = np.array(test_y.tolist())

train_lables_format = [train_labels[i:i+1] for i in range(0, len(train_labels), 1)]    
test_lables_format = [test_labels[i:i+1] for i in range(0, len(test_labels), 1)]

train_y_neural = lb.transform(train_lables_format)
test_y_neural = lb.transform(test_lables_format)

In [102]:
from Text_CNN import TextCNN

# CNN Model
# Leveraged Denny Britz template for CNN:
# http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/

# run this to clear tf flags without re-starting kernel

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
del_all_flags(tf.flags.FLAGS)

# Parameters
# ==================================================

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.app.flags.DEFINE_string('f', '', 'kernel')

FLAGS = tf.flags.FLAGS
FLAGS(sys.argv)
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================

x_train = np.array(train_x_neural)
y_train = np.array(train_y_neural)
x_dev = np.array(test_x_neural)
y_dev = np.array(test_y_neural)

print("x_train", np.shape(x_train))
print("y_train", np.shape(y_train))
print("x_dev", np.shape(x_dev))
print("x_dev", np.shape(y_dev))

print("Vocabulary Size: {:d}".format(vocab.size))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))


# Training
# ==================================================

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]
            
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=2,
            vocab_size=vocab.size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))




Parameters:
ALLOW_SOFT_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7fa40fa566d8>
BATCH_SIZE=<absl.flags._flag.Flag object at 0x7fa40fa56c88>
CHECKPOINT_EVERY=<absl.flags._flag.Flag object at 0x7fa40fa56828>
DROPOUT_KEEP_PROB=<absl.flags._flag.Flag object at 0x7fa40fa56b70>
EMBEDDING_DIM=<absl.flags._flag.Flag object at 0x7fa40fa56cc0>
EVALUATE_EVERY=<absl.flags._flag.Flag object at 0x7fa40fa568d0>
F=<absl.flags._flag.Flag object at 0x7fa40fa50dd8>
FILTER_SIZES=<absl.flags._flag.Flag object at 0x7fa40fa56da0>
L2_REG_LAMBDA=<absl.flags._flag.Flag object at 0x7fa40fa56b38>
LOG_DEVICE_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7fa40fa566a0>
NUM_CHECKPOINTS=<absl.flags._flag.Flag object at 0x7fa40fa56780>
NUM_EPOCHS=<absl.flags._flag.Flag object at 0x7fa40fa56be0>
NUM_FILTERS=<absl.flags._flag.Flag object at 0x7fa40fa56d68>

x_train (830, 71)
y_train (830, 2)
x_dev (209, 71)
x_dev (209, 2)
Vocabulary Size: 1699
Train/Dev split: 830/209
INFO:tensorflow:Summary name embeddin

2018-08-10T17:23:00.315725: step 83, loss 0.238922, acc 0.921875
2018-08-10T17:23:00.463899: step 84, loss 0.208937, acc 0.890625
2018-08-10T17:23:00.608989: step 85, loss 0.223871, acc 0.90625
2018-08-10T17:23:00.755720: step 86, loss 0.335182, acc 0.921875
2018-08-10T17:23:00.904718: step 87, loss 0.420747, acc 0.890625
2018-08-10T17:23:01.050558: step 88, loss 0.212601, acc 0.90625
2018-08-10T17:23:01.196805: step 89, loss 0.270165, acc 0.890625
2018-08-10T17:23:01.342304: step 90, loss 0.28581, acc 0.859375
2018-08-10T17:23:01.490378: step 91, loss 0.233482, acc 0.903226
2018-08-10T17:23:01.638439: step 92, loss 0.134244, acc 0.953125
2018-08-10T17:23:01.784425: step 93, loss 0.324402, acc 0.84375
2018-08-10T17:23:01.929684: step 94, loss 0.367473, acc 0.859375
2018-08-10T17:23:02.073732: step 95, loss 0.325345, acc 0.90625
2018-08-10T17:23:02.218620: step 96, loss 0.334747, acc 0.9375
2018-08-10T17:23:02.363697: step 97, loss 0.448483, acc 0.859375
2018-08-10T17:23:02.509236: step

2018-08-10T17:23:18.052475: step 203, loss 0.0476842, acc 0.984375
2018-08-10T17:23:18.197471: step 204, loss 0.078586, acc 0.96875
2018-08-10T17:23:18.344367: step 205, loss 0.101268, acc 0.9375
2018-08-10T17:23:18.489857: step 206, loss 0.105888, acc 0.96875
2018-08-10T17:23:18.636840: step 207, loss 0.22578, acc 0.890625
2018-08-10T17:23:18.781699: step 208, loss 0.12881, acc 0.983871
2018-08-10T17:23:18.929561: step 209, loss 0.0964404, acc 0.96875
2018-08-10T17:23:19.075859: step 210, loss 0.160909, acc 0.953125
2018-08-10T17:23:19.221835: step 211, loss 0.0839287, acc 0.953125
2018-08-10T17:23:19.366257: step 212, loss 0.158134, acc 0.953125
2018-08-10T17:23:19.514031: step 213, loss 0.0542743, acc 0.96875
2018-08-10T17:23:19.660571: step 214, loss 0.105081, acc 0.953125
2018-08-10T17:23:19.805278: step 215, loss 0.129346, acc 0.96875
2018-08-10T17:23:19.950824: step 216, loss 0.0509261, acc 0.984375
2018-08-10T17:23:20.097222: step 217, loss 0.0937956, acc 0.96875
2018-08-10T17:

2018-08-10T17:23:36.067999: step 326, loss 0.0332118, acc 0.96875
2018-08-10T17:23:36.216288: step 327, loss 0.0262106, acc 1
2018-08-10T17:23:36.362472: step 328, loss 0.0736995, acc 0.984375
2018-08-10T17:23:36.507713: step 329, loss 0.0563345, acc 0.984375
2018-08-10T17:23:36.651347: step 330, loss 0.0330186, acc 0.984375
2018-08-10T17:23:36.800402: step 331, loss 0.0219713, acc 1
2018-08-10T17:23:36.949080: step 332, loss 0.027766, acc 0.984375
2018-08-10T17:23:37.094419: step 333, loss 0.0538668, acc 0.984375
2018-08-10T17:23:37.239483: step 334, loss 0.00811313, acc 1
2018-08-10T17:23:37.389980: step 335, loss 0.0583028, acc 0.984375
2018-08-10T17:23:37.555818: step 336, loss 0.0512831, acc 0.96875
2018-08-10T17:23:37.705890: step 337, loss 0.00685048, acc 1
2018-08-10T17:23:37.848636: step 338, loss 0.0117536, acc 1
2018-08-10T17:23:37.995786: step 339, loss 0.0342913, acc 0.96875
2018-08-10T17:23:38.141942: step 340, loss 0.0575896, acc 0.984375
2018-08-10T17:23:38.287423: step

2018-08-10T17:23:54.687218: step 453, loss 0.0315499, acc 0.984375
2018-08-10T17:23:54.828351: step 454, loss 0.0123577, acc 1
2018-08-10T17:23:54.965400: step 455, loss 0.0347902, acc 0.983871
2018-08-10T17:23:55.108780: step 456, loss 0.0195752, acc 1
2018-08-10T17:23:55.253479: step 457, loss 0.00678563, acc 1
2018-08-10T17:23:55.400627: step 458, loss 0.016621, acc 1
2018-08-10T17:23:55.549114: step 459, loss 0.0427313, acc 0.96875
2018-08-10T17:23:55.697059: step 460, loss 0.0206861, acc 1
2018-08-10T17:23:55.846967: step 461, loss 0.0179563, acc 1
2018-08-10T17:23:55.994099: step 462, loss 0.038201, acc 0.984375
2018-08-10T17:23:56.141767: step 463, loss 0.0172008, acc 1
2018-08-10T17:23:56.289428: step 464, loss 0.0038178, acc 1
2018-08-10T17:23:56.439291: step 465, loss 0.0195297, acc 1
2018-08-10T17:23:56.590715: step 466, loss 0.0110514, acc 1
2018-08-10T17:23:56.740793: step 467, loss 0.017281, acc 0.984375
2018-08-10T17:23:56.884740: step 468, loss 0.00589238, acc 1
2018-08

...

2018-08-10T17:27:42.618426: step 1989, loss 0.000156628, acc 1
2018-08-10T17:27:42.757799: step 1990, loss 0.000902184, acc 1
2018-08-10T17:27:42.898439: step 1991, loss 0.000121217, acc 1
2018-08-10T17:27:43.036057: step 1992, loss 0.00170746, acc 1
2018-08-10T17:27:43.180239: step 1993, loss 0.000937068, acc 1
2018-08-10T17:27:43.318471: step 1994, loss 0.000809605, acc 1
2018-08-10T17:27:43.463003: step 1995, loss 0.000327368, acc 1
2018-08-10T17:27:43.599426: step 1996, loss 0.000103281, acc 1
2018-08-10T17:27:43.736032: step 1997, loss 0.000207937, acc 1
2018-08-10T17:27:43.872793: step 1998, loss 0.0105728, acc 1
2018-08-10T17:27:44.011228: step 1999, loss 4.73496e-05, acc 1
2018-08-10T17:27:44.151112: step 2000, loss 0.000491394, acc 1

Evaluation:
2018-08-10T17:27:44.292089: step 2000, loss 0.930001, acc 0.851675

Saved model checkpoint to /home/mfrazzini/W266_project/deep_learning_sentiment_classification/runs/1533921767/checkpoints/model-2000

2018-08-10T17:27:44.481510: step

2018-08-10T17:28:01.067398: step 2113, loss 0.000135452, acc 1
2018-08-10T17:28:01.213872: step 2114, loss 0.000630985, acc 1
2018-08-10T17:28:01.360322: step 2115, loss 0.000104279, acc 1
2018-08-10T17:28:01.512619: step 2116, loss 0.00149036, acc 1
2018-08-10T17:28:01.665763: step 2117, loss 0.000114499, acc 1
2018-08-10T17:28:01.810741: step 2118, loss 9.78439e-05, acc 1
2018-08-10T17:28:01.957424: step 2119, loss 0.00198683, acc 1
2018-08-10T17:28:02.112030: step 2120, loss 0.0220019, acc 0.984375
2018-08-10T17:28:02.256969: step 2121, loss 7.90759e-05, acc 1
2018-08-10T17:28:02.398420: step 2122, loss 0.000346047, acc 1
2018-08-10T17:28:02.538225: step 2123, loss 0.000691245, acc 1
2018-08-10T17:28:02.679309: step 2124, loss 0.000441069, acc 1
2018-08-10T17:28:02.819809: step 2125, loss 0.000229023, acc 1
2018-08-10T17:28:02.958917: step 2126, loss 0.00201864, acc 1
2018-08-10T17:28:03.098301: step 2127, loss 5.57895e-05, acc 1
2018-08-10T17:28:03.237068: step 2128, loss 0.0001644

2018-08-10T17:28:19.940596: step 2241, loss 1.55085e-05, acc 1
2018-08-10T17:28:20.087993: step 2242, loss 0.00101146, acc 1
2018-08-10T17:28:20.240287: step 2243, loss 0.000347452, acc 1
2018-08-10T17:28:20.389532: step 2244, loss 7.09535e-05, acc 1
2018-08-10T17:28:20.538615: step 2245, loss 0.00211001, acc 1
2018-08-10T17:28:20.687590: step 2246, loss 7.70882e-05, acc 1
2018-08-10T17:28:20.836781: step 2247, loss 0.00135503, acc 1
2018-08-10T17:28:20.984766: step 2248, loss 0.000254837, acc 1
2018-08-10T17:28:21.132587: step 2249, loss 6.18753e-05, acc 1
2018-08-10T17:28:21.286673: step 2250, loss 5.18689e-05, acc 1
2018-08-10T17:28:21.446598: step 2251, loss 0.000206844, acc 1
2018-08-10T17:28:21.595676: step 2252, loss 0.000248282, acc 1
2018-08-10T17:28:21.746465: step 2253, loss 0.000296266, acc 1
2018-08-10T17:28:21.893880: step 2254, loss 0.000112172, acc 1
2018-08-10T17:28:22.044348: step 2255, loss 2.13968e-05, acc 1
2018-08-10T17:28:22.196078: step 2256, loss 0.0067306, acc

2018-08-10T17:28:38.881113: step 2370, loss 0.000560287, acc 1
2018-08-10T17:28:39.026767: step 2371, loss 2.38795e-05, acc 1
2018-08-10T17:28:39.173355: step 2372, loss 4.7792e-05, acc 1
2018-08-10T17:28:39.318383: step 2373, loss 1.28101e-05, acc 1
2018-08-10T17:28:39.468621: step 2374, loss 0.00023922, acc 1
2018-08-10T17:28:39.616196: step 2375, loss 0.000259394, acc 1
2018-08-10T17:28:39.760949: step 2376, loss 7.42448e-05, acc 1
2018-08-10T17:28:39.906572: step 2377, loss 0.000112914, acc 1
2018-08-10T17:28:40.054656: step 2378, loss 0.000120646, acc 1
2018-08-10T17:28:40.198634: step 2379, loss 0.00282183, acc 1
2018-08-10T17:28:40.346853: step 2380, loss 5.01917e-05, acc 1
2018-08-10T17:28:40.497101: step 2381, loss 3.59057e-05, acc 1
2018-08-10T17:28:40.651481: step 2382, loss 0.000389609, acc 1
2018-08-10T17:28:40.800794: step 2383, loss 0.00114749, acc 1
2018-08-10T17:28:40.948970: step 2384, loss 0.00012051, acc 1
2018-08-10T17:28:41.101069: step 2385, loss 7.81602e-05, acc

2018-08-10T17:28:57.635726: step 2497, loss 0.00431111, acc 1
2018-08-10T17:28:57.783131: step 2498, loss 7.47595e-05, acc 1
2018-08-10T17:28:57.929861: step 2499, loss 7.04662e-05, acc 1
2018-08-10T17:28:58.074015: step 2500, loss 0.006382, acc 1

Evaluation:
2018-08-10T17:28:58.214230: step 2500, loss 0.978796, acc 0.842105

Saved model checkpoint to /home/mfrazzini/W266_project/deep_learning_sentiment_classification/runs/1533921767/checkpoints/model-2500

2018-08-10T17:28:58.417247: step 2501, loss 0.00114533, acc 1
2018-08-10T17:28:58.565601: step 2502, loss 0.000156147, acc 1
2018-08-10T17:28:58.714256: step 2503, loss 0.00111408, acc 1
2018-08-10T17:28:58.854348: step 2504, loss 0.000137951, acc 1
2018-08-10T17:28:58.994436: step 2505, loss 0.000620643, acc 1
2018-08-10T17:28:59.136146: step 2506, loss 0.0049684, acc 1
2018-08-10T17:28:59.281683: step 2507, loss 8.86211e-05, acc 1
2018-08-10T17:28:59.430902: step 2508, loss 0.000367905, acc 1
2018-08-10T17:28:59.578208: step 2509